In [2]:
import numpy as np
import pandas as pd
import vcf, glob, os, shutil, re, time
from Bio import Seq, SeqIO, Entrez

db_name = 'NTM_MTBC'

In [3]:
output_dir = "/n/data1/hms/dbmi/farhat/rollingDB/genomic_data/HAPL_4JHX48QMO4685KC6/HAPL_4JHX48QMO4685KC6/kraken"
os.listdir(output_dir)

['HAPL_4JHX48QMO4685KC6_1.kraken.filtered.fastq.gz',
 'HAPL_4JHX48QMO4685KC6_2.kraken.filtered.fastq.gz',
 'kraken_classifications',
 'kraken_classifications_standard_DB.txt.gz',
 'kraken_report',
 'kraken_report_standard_DB.txt',
 'unclassified_percent.txt']

In [4]:
339268, 1790, 1780

inf

In [ ]:
grep '^>' /n/scratch/users/s/sak0914/mycobacteriaceae_genomes_renamed/*.fna | grep -E '\|kraken\|(339268|1790|1780)'

In [8]:
genomes = glob.glob("/n/scratch/users/s/sak0914/mycobacteriaceae_genomes_renamed/*")
len(genomes)

14438

In [ ]:
# for i, genome in enumerate(genomes):
#     fasta_headers = [seq.id for seq in SeqIO.parse(genome, "fasta")]

#     for header in fasta_headers:
#         if '339268' in header:
#             print(genome, '339268')
#             break
#         elif '1790' in header:
#             print(genome, '1790')
#             break
#         elif '1780' in header:
#             print(genome, '1780')
#             break

#     if i % 1000 == 0:
#         print(i)

# Get MTBC and NTM Taxids

In [2]:
# # M. abscessus is Mycobacteroides, a different genus. So include everything within the family Mycobacteriaceae (1762) 
# all_mycobacteria_taxids = get_child_taxids(1762, "skulkarni@g.harvard.edu")
# # print(f"{len(all_MTBC_taxids)} MTBC taxids")
# print(f"{len(all_mycobacteria_taxids)} mycobacterial (including NTM) taxids")
# pd.Series(all_mycobacteria_taxids).to_csv(f"./Mycobacteriaceae_taxids.csv", index=False, header=None)

In [2]:
all_mycobacteria_taxids = pd.read_csv(f"./Mycobacteriaceae_taxids.csv", header=None)[0].values
print(1762 in all_mycobacteria_taxids)
print(f"{len(all_mycobacteria_taxids)} mycobacterial (including NTM) taxids")

True
7475 mycobacterial (including NTM) taxids


# Get NCBI genomes used to construct the standard kraken database, but only within the family

## 1. Download Genomes from NCBI

In [ ]:
# wget ftp://ftp.ncbi.nih.gov/genomes/genbank/bacteria/assembly_summary.txt

In [25]:
bacteria_assembly_summary = pd.read_csv("/n/scratch/users/s/sak0914/bacteria_assembly_summary.txt", sep='\t', skiprows=1)

/tmp/ipykernel_5562/180699650.py:1: DtypeWarning: Columns (34,35,36) have mixed types. Specify dtype option on import or set low_memory=False.
  bacteria_assembly_summary = pd.read_csv("/n/scratch/users/s/sak0914/bacteria_assembly_summary.txt", sep='\t', skiprows=1)


In [36]:
found_taxids = bacteria_assembly_summary.query("taxid in @all_mycobacteria_taxids").taxid.unique()
len(found_taxids)

2635

In [46]:
bacteria_assembly_summary.query("taxid==1762 | species_taxid == 1762")

,#assembly_accession,bioproject,biosample,wgs_master,refseq_category,taxid,species_taxid,organism_name,infraspecific_name,isolate,...,replicon_count,scaffold_count,contig_count,annotation_provider,annotation_name,annotation_date,total_gene_count,protein_coding_gene_count,non_coding_gene_count,pubmed_id


In [44]:
77643 in found_taxids, 1765 in found_taxids, 78331 in found_taxids, 1762 in found_taxids

(False, True, True, False)

In [49]:
# check haha
bacteria_assembly_summary.query("taxid in @all_mycobacteria_taxids & ~organism_name.str.contains('Myco')")

,#assembly_accession,bioproject,biosample,wgs_master,refseq_category,taxid,species_taxid,organism_name,infraspecific_name,isolate,...,replicon_count,scaffold_count,contig_count,annotation_provider,annotation_name,annotation_date,total_gene_count,protein_coding_gene_count,non_coding_gene_count,pubmed_id


In [126]:
bacteria_assembly_summary.query("(taxid in @all_mycobacteria_taxids | species_taxid in @all_mycobacteria_taxids) & ftp_path != 'na'")[['#assembly_accession', 'ftp_path', 'taxid', 'organism_name']].dropna().to_csv("./Mycobacteriaceae_assemblies.txt", sep='\t', index=False, header=None)

In [4]:
mycobacteria_assemblies = pd.read_csv("./Mycobacteriaceae_assemblies.txt", sep='\t', header=None)
mycobacteria_assemblies.columns = ['Assembly', 'FTP_path', 'TaxID', 'Organism_Name']
mycobacteria_assemblies['SampleName'] = [os.path.basename(FTP_path) for FTP_path in mycobacteria_assemblies['FTP_path'].values]
len(mycobacteria_assemblies)

14438

In [5]:
mycobacteria_assemblies.TaxID.nunique()

2636

In [8]:
292462 in mycobacteria_assemblies.TaxID.values

True

In [9]:
mycobacteria_assemblies.query("TaxID==292462")

,Assembly,FTP_path,TaxID,Organism_Name,SampleName
9748,GCA_002101635.1,https://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/0...,292462,Mycobacterium florentinum,GCA_002101635.1_ASM210163v1
9749,GCA_010730355.1,https://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/0...,292462,Mycobacterium florentinum,GCA_010730355.1_ASM1073035v1
9750,GCA_025823305.1,https://ftp.ncbi.nlm.nih.gov/genomes/all/GCA/0...,292462,Mycobacterium florentinum,GCA_025823305.1_ASM2582330v1


```bash
source activate bioinformatics

out_dir="/n/scratch/users/s/sak0914/mycobacteriaceae_genomes"

while IFS=$'\t', read -r accession ftp_path taxid oranism_name; do
    
    sampleName=$(basename "$ftp_path")

    if [ ! -f "$out_dir/${sampleName}_genomic.fna" ]; then
        wget "$ftp_path/${sampleName}_genomic.fna.gz" -P $out_dir
        gunzip "$out_dir/${sampleName}_genomic.fna.gz"

    fi
    
done < /home/sak0914/Mtb_Megapipe/kraken_db/NTM_MTBC/Mycobacteriaceae_assemblies.txt
```

## 2. Remove plasmid sequences because they are shared across NTMs and can confuse the classification

We don't need the sensitivity of plasmids because MTBC doesn't have plasmids. So the contaminating reads that we're trying to filter out won't map to MTBC anyway because they're too divergent.

In [28]:
# in_genomes_dir = "/n/scratch/users/s/sak0914/mycobacteriaceae_genomes"
# out_genomes_dir = "/n/scratch/users/s/sak0914/mycobacteriaceae_no_plasmids"

# os.makedirs(out_genomes_dir, exist_ok=True)

# for infile in glob.glob(f"{in_genomes_dir}/*.fna"):

#     outfile = os.path.join(out_genomes_dir, os.path.basename(infile))

#     all_records = [seq for seq in SeqIO.parse(infile, "fasta")]
#     records = [seq for seq in SeqIO.parse(infile, "fasta") if "plasmid" not in seq.description]
    
#     # Write the non-plasmid records to the output file
#     SeqIO.write(records, outfile, "fasta")

#     # Optionally, print plasmid sequences found
#     if len(all_records) > len(records):
#         print(f"{infile} has {len(all_records) - len(records)} plasmids")

## 3. Rename headers of the FASTQ files to include the Taxids

In [11]:
# in_genomes_dir = "/n/scratch/users/s/sak0914/mycobacteriaceae_no_plasmids"
# out_genomes_dir = "/n/scratch/users/s/sak0914/mycobacteriaceae_genomes_renamed"

# unique_taxids = []

# for i, fName in enumerate(glob.glob(f"{in_genomes_dir}/*")):

#     out_fName = os.path.join(out_genomes_dir, os.path.basename(fName))

#     if os.path.isfile(fName): # not os.path.isfile(out_fName) and 

#         assembly_name = os.path.basename(fName).split('_genomic')[0]
    
#         found_name = mycobacteria_assemblies.query("FTP_path.str.contains(@assembly_name)")
#         assert len(found_name) == 1
        
#         taxid = found_name.TaxID.values[0]
#         unique_taxids.append(taxid)

#         # exclude the description for now
#         sequences = [(seq.id, seq.seq) for seq in SeqIO.parse(fName, "fasta")]
    
#         # there could be plasmids or multiple contigs because NTMs have plasmids, or the genomes may not be amazing so they are fragmented into multiple contigs so there will be multiple sequences per FASTA file
#         # assert len(sequences) == 1
    
#         # write to a new directory so that we don't change the existing FASTQ files
#         with open(out_fName, "w+") as file:
            
#             # write all the sequences back
#             for seq in sequences:
                
#                 # append the taxid to the existing sequence name
#                 header = seq[0] + f'|kraken:taxid|{taxid}'# + seq[1]
                
#                 file.write(f">{header}\n")
#                 file.write(str(seq[1]) + "\n")

#     if i % 1000 == 0:
#         print(i)

# unique_taxids = np.unique(unique_taxids)

In [5]:
len(unique_taxids), mycobacteria_assemblies.TaxID.nunique()

(2636, 2636)

In [6]:
set(unique_taxids) - set(mycobacteria_assemblies.TaxID)

set()

In [ ]:
DBNAME="mycobacteriaceae"

kraken2-build --download-taxonomy --db $DBNAME

for file in mycobacteriaceae_genomes_renamed/*.fna
do
    kraken2-build --add-to-library $file --db $DBNAME
done

# build the database. Took approximately 5 minutes for 155 genomes
kraken2-build --build --db $DBNAME

# verify that the database was built correctly
kraken2-inspect --db $DBNAME

# clean the database to reduce disk usage
# kraken2-build --clean --db $DBNAME

# Use the Kraken2 NTM/MTBC database and adjust the confidence parameter to tune accuracy

## Compute the density of unfixed SNPs (maybe in essential genes only?) to determine accuracy. High = lots of NTM reads

In [68]:
DBNAME = "/n/scratch/users/s/sak0914/mycobacteriaceae"

input_dir = "/n/data1/hms/dbmi/farhat/rollingDB/genomic_data"
output_dir = "/n/scratch/users/s/sak0914/kraken_DB_tuning"

sample = "HAPL_4JHX48QMO4685KC6"

FQ1 = f"{input_dir}/{sample}/{sample}/fastp/{sample}.R1.trimmed.fastq.gz"
FQ2 = f"{input_dir}/{sample}/{sample}/fastp/{sample}.R2.trimmed.fastq.gz"

kraken_report = f"{output_dir}/{sample}/{sample}/kraken/kraken_NTM_MTBC_report.txt"
kraken_classifications = f"{output_dir}/{sample}/{sample}/kraken/kraken_NTM_MTBC_classifications.txt"

In [80]:
df_kraken_classifications = pd.read_csv(kraken_classifications, sep='\t', header=None)
df_kraken_classifications.columns = ['Classified', 'ReadName', 'TaxID', 'Length', 'LCA']

In [105]:
read_name = 'A01739:164:H7HNNDSXF:3:1576:19913:7138'

read_seq = 'GTCTGTCGCGTGAGCGCGCGGGCCTCGAGGTTCGTGACGTGCACCCGTCGCACTACGGCCGGATGTGTCCGATCGAGACTCCTGAGGGTCCGAACATCGGTCTGATCGGGTCGTTGTCGGTGTACGCACGGGTCAACCCGTTCGGGTTCATCGAGACGC'

In [82]:
df_kraken_classifications.query("ReadName==@read_name").LCA.values

array(['1762:42 1763:2 1762:1 1763:5 1762:14 1763:1 1762:20 1763:1 1762:38 1763:1 |:| 1763:8 1762:3 1763:1 1762:9 141349:7 1762:9 141349:9 1762:1 141349:1 1762:9 1763:5 141349:2 1762:39 1763:1 1762:5 1763:16'],
      dtype=object)

In [89]:
df_kraken_classifications.query("ReadName==@read_name")

,Classified,ReadName,TaxID,Length,LCA
994005,C,A01739:164:H7HNNDSXF:3:1576:19913:7138,1762,159|159,1762:42 1763:2 1762:1 1763:5 1762:14 1763:1 17...


In [76]:
df_kraken_classifications.query("ReadName==@read_name").LCA.values

array(['1762:42 1763:2 1762:1 1763:5 1762:14 1763:1 1762:20 1763:1 1762:38 1763:1 |:| 1763:8 1762:3 1763:1 1762:9 141349:7 1762:9 141349:9 1762:1 141349:1 1762:9 1763:5 141349:2 1762:39 1763:1 1762:5 1763:16'],
      dtype=object)

In [ ]:
# blastn the FASTA file. -outfmt 6 is output in tabular format
echo -e ">A01739:164:H7HNNDSXF:3:1576:19913:7138\nGTCTGTCGCGTGAGCGCGCGGGCCTCGAGGTTCGTGACGTGCACCCGTCGCACTACGGCCGGATGTGTCCGATCGAGACTCCTGAGGGTCCGAACATCGGTCTGATCGGGTCGTTGTCGGTGTACGCACGGGTCAACCCGTTCGGGTTCATCGAGACGC" | blastn -max_target_seqs 10 -task megablast -query - -db /n/scratch/users/s/sak0914/mycobacteriaceae_blast_DB/mycobacteriaceae_blast_DB -outfmt 6 -out /home/sak0914/result.txt

In [4]:
blastn_columns = [
    "qseqid",    # Query sequence ID
    "sseqid",    # Subject (database) sequence ID
    "pident",    # Percentage of identical matches
    "length",    # Alignment length
    "mismatch",  # Number of mismatches
    "gapopen",   # Number of gap openings
    "qstart",    # Start of alignment in query
    "qend",      # End of alignment in query
    "sstart",    # Start of alignment in subject
    "send",      # End of alignment in subject
    "evalue",    # Expectation value (E-value)
    "bitscore"   # Bit score
]

In [9]:
# it's already sorted
blast_result = pd.read_csv("/home/sak0914/result.txt", sep="\t", header=None, names=blastn_columns).sort_values("bitscore", ascending=False)

blast_result['TaxID'] = blast_result['sseqid'].str.split('|').str[-1].astype(int)

top_taxon = blast_result.TaxID.values[0]
print(top_taxon)

141349


In [11]:
blast_result

,qseqid,sseqid,pident,length,mismatch,gapopen,qstart,qend,sstart,send,evalue,bitscore,TaxID
0,A01739:164:H7HNNDSXF:3:1576:19913:7138,CP092423.2|kraken:taxid|141349,100.000,159,0,0,1,159,933184,933342,1.650000e-76,294,141349
1,A01739:164:H7HNNDSXF:3:1576:19913:7138,CTEE01000001.1|kraken:taxid|141349,100.000,159,0,0,1,159,1127957,1128115,1.650000e-76,294,141349
2,A01739:164:H7HNNDSXF:3:1576:19913:7138,FXEG02000002.1|kraken:taxid|1964395,97.484,159,4,0,1,159,423154,423312,7.730000e-70,272,1964395
3,A01739:164:H7HNNDSXF:3:1576:19913:7138,JBCHKV010000001.1|kraken:taxid|3142488,96.855,159,5,0,1,159,218276,218118,3.600000e-68,267,3142488
4,A01739:164:H7HNNDSXF:3:1576:19913:7138,JAZGLW010000001.1|kraken:taxid|3096112,96.855,159,5,0,1,159,599943,599785,3.600000e-68,267,3096112
5,A01739:164:H7HNNDSXF:3:1576:19913:7138,JACKVN010000013.1|kraken:taxid|292462,96.855,159,5,0,1,159,49216,49058,3.600000e-68,267,292462
6,A01739:164:H7HNNDSXF:3:1576:19913:7138,BQYH01000027.1|kraken:taxid|154654,96.855,159,5,0,1,159,367324,367482,3.600000e-68,267,154654
7,A01739:164:H7HNNDSXF:3:1576:19913:7138,BQYG01000056.1|kraken:taxid|154654,96.855,159,5,0,1,159,367356,367514,3.600000e-68,267,154654
8,A01739:164:H7HNNDSXF:3:1576:19913:7138,BQYF01000131.1|kraken:taxid|154654,96.855,159,5,0,1,159,132456,132614,3.600000e-68,267,154654
9,A01739:164:H7HNNDSXF:3:1576:19913:7138,BQYE01000035.1|kraken:taxid|154654,96.855,159,5,0,1,159,132469,132627,3.600000e-68,267,154654


In [ ]:
cut -f1 /n/scratch/users/s/sak0914/kraken_DB_tuning/HAPL_4JHX48QMO4685KC6/HAPL_4JHX48QMO4685KC6/BLAST/hits.tsv | sort | uniq | wc -l


In [18]:
blast_result_full = pd.read_csv("/n/scratch/users/s/sak0914/kraken_DB_tuning/HAPL_4JHX48QMO4685KC6/HAPL_4JHX48QMO4685KC6/BLAST/hits.tsv", sep='\t', header=None, names= blastn_columns).sort_values("bitscore", ascending=False)

blast_result_full['TaxID'] = blast_result_full['sseqid'].str.split('|').str[-1].astype(int)

# need to do 550k total
print(blast_result_full.qseqid.nunique())

552289


In [19]:
top_taxon_per_read = blast_result_full.drop_duplicates('qseqid')[['qseqid', 'TaxID']]

In [21]:
top_taxon_per_read.query("TaxID in [77643, 1773]")

,qseqid,TaxID
0,A01739:164:H7HNNDSXF:3:1101:14760:1063,1773
3529072,A01739:164:H7HNNDSXF:3:2130:28800:22999,1773
3529078,A01739:164:H7HNNDSXF:3:2130:25880:23015,1773
3529086,A01739:164:H7HNNDSXF:3:2130:29803:23109,1773
3529097,A01739:164:H7HNNDSXF:3:2130:31928:23124,1773
...,...,...
4436043,A01739:164:H7HNNDSXF:3:2325:12210:32409,1773
4762916,A01739:164:H7HNNDSXF:3:2403:21730:9157,1773
1946439,A01739:164:H7HNNDSXF:3:1428:26042:35571,1773
4698922,A01739:164:H7HNNDSXF:3:2370:23068:11130,1773


In [31]:
def load_nodes_dmp(nodes_path):
    parent_map = {}
    child_map = {}
    with open(nodes_path, 'r') as f:
        for line in f:
            parts = [x.strip() for x in line.split('|')]
            child_taxid, parent_taxid = int(parts[0]), int(parts[1])
            parent_map[child_taxid] = parent_taxid
            if parent_taxid not in child_map:
                child_map[parent_taxid] = []
            child_map[parent_taxid].append(child_taxid)
    return parent_map, child_map
    

    
def get_child_taxids(taxid, child_map):
    descendants = []
    stack = [taxid]
    while stack:
        current = stack.pop()
        children = child_map.get(current, [])
        descendants.extend(children)
        stack.extend(children)
    return descendants


# analyze taxonomy map 
db_name = "/n/scratch/users/s/sak0914/mycobacteriaceae"
parent_map, child_map = load_nodes_dmp(f"{db_name}/taxonomy/nodes.dmp")

# use the functionss to get all taxids
child_taxids = get_child_taxids(77643, child_map)
len(child_taxids)

3296

In [33]:
top_taxon_per_read.query("TaxID in @child_taxids")['qseqid'].drop_duplicates().to_csv("/n/scratch/users/s/sak0914/kraken_DB_tuning/HAPL_4JHX48QMO4685KC6/HAPL_4JHX48QMO4685KC6/BLAST/MTBC_read_names.txt", sep='\t', header=None, index=False)

In [ ]:
/n/data1/hms/dbmi/farhat/rollingDB/genomic_data/HAPL_4JHX48QMO4685KC6/HAPL_4JHX48QMO4685KC6/fastp/HAPL_4JHX48QMO4685KC6.R1.trimmed.fastq.gz

In [ ]:
cat <(seqtk subseq /n/data1/hms/dbmi/farhat/rollingDB/genomic_data/HAPL_4JHX48QMO4685KC6/HAPL_4JHX48QMO4685KC6/fastp/HAPL_4JHX48QMO4685KC6.R2.trimmed.fastq.gz /n/scratch/users/s/sak0914/kraken_DB_tuning/HAPL_4JHX48QMO4685KC6/HAPL_4JHX48QMO4685KC6/kraken/MTBC_read_names.txt) <(seqtk subseq /n/data1/hms/dbmi/farhat/rollingDB/genomic_data/HAPL_4JHX48QMO4685KC6/HAPL_4JHX48QMO4685KC6/fastp/HAPL_4JHX48QMO4685KC6.R2.trimmed.fastq.gz /n/scratch/users/s/sak0914/kraken_DB_tuning/HAPL_4JHX48QMO4685KC6/HAPL_4JHX48QMO4685KC6/BLAST/MTBC_read_names.txt) > /n/scratch/users/s/sak0914/kraken_DB_tuning/HAPL_4JHX48QMO4685KC6/HAPL_4JHX48QMO4685KC6/bam/HAPL_4JHX48QMO4685KC6.R2.MTBC.fastq

In [2]:
import pandas as pd

In [29]:
df_depth = pd.read_csv("/n/scratch/users/s/sak0914/kraken_DB_tuning/HAPL_4JHX48QMO4685KC6/bam/HAPL_4JHX48QMO4685KC6.depth.tsv.gz", compression='gzip', header=None, sep='\t')
# df_depth = pd.read_csv("/n/data1/hms/dbmi/farhat/rollingDB/genomic_data/HAPL_4JHX48QMO4685KC6/bam/HAPL_4JHX48QMO4685KC6.depth.tsv.gz", compression='gzip', header=None, sep='\t')

In [30]:
df_depth[2].median(), df_depth[2].mean()

(31.0, 31.1845352136174)

In [31]:
len(df_depth.loc[df_depth[2] >= 10]) / len(df_depth)

0.9642044985732847

In [21]:
df_variants = pd.read_csv("/n/scratch/users/s/sak0914/kraken_DB_tuning/HAPL_4JHX48QMO4685KC6/WHO_resistance/HAPL_4JHX48QMO4685KC6_variants_annot.tsv", sep='\t')

In [22]:
df_variants.query("confidence in ['1) Assoc w R', '2) Assoc w R - Interim']")

,POS,REF,ALT,FILTER,QUAL,IMPRECISE,AF,DP,BQ,MQ,IC,DC,EFFECT,GENE,HGVS_C,HGVS_P,variant,drug,confidence
57,2155168,C,G,PASS,1037.0,False,1.0,29,36,60,0,0,missense_variant,katG,c.944G>C,p.Ser315Thr,katG_p.Ser315Thr,Isoniazid,1) Assoc w R


In [26]:
df_variants.query("GENE=='gyrA'").drop_duplicates('variant')

,POS,REF,ALT,FILTER,QUAL,IMPRECISE,AF,DP,BQ,MQ,IC,DC,EFFECT,GENE,HGVS_C,HGVS_P,variant,drug,confidence
4,7362,G,C,PASS,925.0,False,1.00,25,37,60,0,0,missense_variant,gyrA,c.61G>C,p.Glu21Gln,gyrA_p.Glu21Gln,Levofloxacin,5) Not assoc w R
6,7585,G,C,PASS,1257.0,False,1.00,35,36,60,0,0,missense_variant,gyrA,c.284G>C,p.Ser95Thr,gyrA_p.Ser95Thr,Levofloxacin,5) Not assoc w R
8,8452,C,T,PASS,1078.0,False,1.00,34,36,60,0,2,missense_variant,gyrA,c.1151C>T,p.Ala384Val,gyrA_p.Ala384Val,Levofloxacin,5) Not assoc w R
10,8668,C,T,PASS,803.0,False,0.99,25,34,60,0,0,missense_variant,gyrA,c.1367C>T,p.Ala456Val,gyrA_p.Ala456Val,Levofloxacin,3) Uncertain significance
12,9143,T,C,PASS,1554.0,False,1.00,42,37,60,0,0,synonymous_variant,gyrA,c.1842T>C,p.Ile614Ile,gyrA_c.1842T>C,Levofloxacin,5) Not assoc w R
14,9304,G,A,PASS,1554.0,False,1.00,44,35,60,0,0,missense_variant,gyrA,c.2003G>A,p.Gly668Asp,gyrA_p.Gly668Asp,Levofloxacin,5) Not assoc w R


In [32]:
df_WGS = pd.read_csv("/n/data1/hms/dbmi/farhat/rollingDB/TRUST/Illumina_culture_WGS_summary.csv")

In [46]:
df_run = df_WGS.query("Kraken_Unclassified_Percent >= 5 & Kraken_Unclassified_Percent <= 20")[['SampleID', 'SampleID']]
df_run['FQ'] = 0
print(len(df_run))
df_run.to_csv("/home/sak0914/Mtb_Megapipe/isolates_to_run_kraken.tsv", sep='\t', header=None, index=False)

6


In [48]:
df_WGS.query("Kraken_Unclassified_Percent >= 5 & Kraken_Unclassified_Percent <= 20")

,SampleID,Kraken_Unclassified_Percent,Mean_Depth,Median_Depth,Perc_Sites_10x,Perc_Sites_20x,F2,Coll2014,Freschi2020,Lipworth2019,Shitikov2017,Stucki2016,Lineage,Freschi_Lineage_1,Freschi_Lineage_2,Culture_Mixed_Infection
43,MFS-138,7.09,292.666505,300.0,98.507775,98.362814,0.010892,2.2.1.1,2.2.1.1.1,beijing,"lin2.2.1,pacific_RD150",NaN,2,2.2.1.1.1,NaN,0
98,MFS-188,17.48,233.911604,239.0,98.437935,98.264072,0.011793,2.2.1,2.2.1.1.1,beijing,"lin2.2.1,asian_african_2",NaN,2,2.2.1.1.1,NaN,0
232,MFS-366,13.22,312.759877,316.0,98.901175,98.767186,0.007009,4.8,4.2.1.1.1.1.1.1.i1,NaN,NaN,4.10/PGG3,4,4.2.1.1.1.1.1.1.i1,NaN,0
274,MFS-409,10.60,263.977886,270.0,98.509157,98.351910,0.015020,2.2.1.1,2.2.1.1.1,"beijing,mungi","lin2.2.1,pacific_RD150",NaN,2,2.2.1.1.1,NaN,0
486,MFS-605,18.03,325.947942,310.0,98.965507,98.838340,0.012019,4.1.1.1,4.1.i1.2.1,"xtype,canetti",NaN,4,4,4.1.i1.2.1,NaN,0
669,MFS-778,12.82,285.274752,292.0,98.679189,98.533004,0.011717,2.2.1.1,2.2.1.1.1,"beijing,mungi","lin2.2.1,pacific_RD150",NaN,2,2.2.1.1.1,NaN,0
